In [ ]:
#기본 모듈
import numpy as np
import pandas as pd
from tqdm import tqdm

#전처리 모듈
from sklearn.preprocessing import RobustScaler

#모델 평가
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import pearsonr

#Pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import os

In [ ]:
torch.cuda.is_available()

In [ ]:
#데이터 적재
data = pd.read_csv("../Data/electric_train.csv")
data.head()

## Functions

In [ ]:
#컬럼명 변경
def change_column_name(data):
    data.rename(columns={'electric_train.tm' : 'TM',
                    'electric_train.num' : 'NUM',
                     'electric_train.hh24' : 'HH24',
                     'electric_train.stn' : 'STN',
                     'electric_train.sum_load' : 'sum_load',
                     'electric_train.nph_ta' : 'nph_ta',
                     'electric_train.nph_hm' : 'nph_hm',
                     'electric_train.nph_ws_10m' : 'nph_ws_10m',
                     'electric_train.nph_rn_60m' : 'nph_rn_60m',
                     'electric_train.nph_ta_chi' : 'nph_ta_chi',
                     'electric_train.weekday' : 'weekday',
                     'electric_train.week_name' : 'week_name',
                     'electric_train.elec' : 'elect'}, inplace=True)
    return data

In [ ]:
#시간 파생변수
def preprocess_time(data):
    data['TM'] = pd.to_datetime(data['TM']) #datetime 형식으로 변환
    
    #시간 순으로 데이터 정렬
    data = data.sort_values(by='TM')

    data['YY'] = data['TM'].dt.year #년을 새로운 열로 추가
    data['MM'] = data['TM'].dt.month #월을 새로운 열로 추가
    data['DD'] = data['TM'].dt.day #일을 새로운 열로 추가
    return data

In [ ]:
#결측치 처리하기
def fillnan(data):
    #결측치를 nan으로 수정
    data.replace(-99, np.nan, inplace=True) 

    #시계열 데이터를 인덱스로 지정
    data.set_index('TM', inplace=True)

    #선형보간법으로 결측치 처리
    data.interpolate(method='time', inplace=True)

    #인덱스 되돌리기
    data.reset_index(drop=True, inplace=True)

    return data

In [ ]:
#계절 레이블 생성
def create_season_label(data):
    input = data.copy()

    #7월과 8월일 때 1, 그렇지 않을 때 0로 계절 레이블 생성
    input['season_label'] = input['MM'].isin([7, 8]).astype(int)

    return input

In [ ]:
def moving_average_decomposition(data, window=24):
    input = data.copy()

    # 결과를 저장할 열 추가
    input['trend_s'] = 0.0
    input['seasonal_s'] = 0.0
    input['residual_s'] = 0.0

    # 고유한 시즌 라벨에 대해 반복
    for label in input['season_label'].unique():
        subset = input[input['season_label'] == label].copy()
        
        # 이동 평균을 이용한 추세 추정
        subset['trend_s'] = subset['elect'].rolling(window=window, center=True).mean()
        subset['trend_s'] = subset['trend_s'].ffill()
        subset['trend_s'] = subset['trend_s'].bfill()  # 앞뒤 NaN 값 채우기
        
        # 계절성 추정 (원래 값에서 추세를 뺀 값)
        subset['seasonal_s'] = subset['elect'] - subset['trend_s']
        subset['seasonal_s'] = subset['seasonal_s'].ffill()
        subset['seasonal_s'] = subset['seasonal_s'].bfill()  # 앞뒤 NaN 값 채우기
        
        # 잔차 계산 (원래 값에서 추세와 계절성을 뺀 값)
        subset['residual_s'] = subset['elect'] - subset['trend_s'] - subset['seasonal_s']
        subset['residual_s'] = subset['residual_s'].ffill()
        subset['residual_s'] = subset['residual_s'].bfill()  # 앞뒤 NaN 값 채우기
        
        # 원래 데이터 프레임에 결과 할당
        input.loc[subset.index, 'trend_s'] = subset['trend_s']
        input.loc[subset.index, 'seasonal_s'] = subset['seasonal_s']
        input.loc[subset.index, 'residual_s'] = subset['residual_s']

    return input

In [ ]:
#차분값 파생변수
def diff_data(data):
    #전력 차분
    data['difference1_e'] = data['elect'].diff() #1차 차분
    data['difference2_e'] = data['elect'].diff(2) #2차 차분
    data['difference3_e'] = data['elect'].diff(3) #3차 차분

    #습도 차분
    data['difference1_h'] = data['nph_hm'].diff() #1차 차분
    data['difference2_h'] = data['nph_hm'].diff(2) #2차 차분
    data['difference3_h'] = data['nph_hm'].diff(3) #3차 차분

    #기온 차분
    data['difference1_t'] = data['nph_ta'].diff() 
    data['difference2_t'] = data['nph_ta'].diff(2) 
    data['difference3_t'] = data['nph_ta'].diff(3) 

    #앞쪽 채움 처리 후 nan 값이 남아있을 경우 뒤쪽 채움으로 처리
    data['difference1_e'] = data['difference1_e'].ffill().bfill()
    data['difference2_e'] = data['difference2_e'].ffill().bfill()
    data['difference3_e'] = data['difference3_e'].ffill().bfill() 

    data['difference1_h'] = data['difference1_h'].ffill().bfill()
    data['difference2_h'] = data['difference2_h'].ffill().bfill()
    data['difference3_h'] = data['difference3_h'].ffill().bfill()
    
    data['difference1_t'] = data['difference1_t'].ffill().bfill()
    data['difference2_t'] = data['difference2_t'].ffill().bfill()
    data['difference3_t'] = data['difference3_t'].ffill().bfill()

    return data

In [ ]:
#scailing
def scailing_data(data, selected_features):
    #스케일러 초기화
    scaler_R = RobustScaler()

    #스케일링이 필요한 컬럼 목록
    S_columns = ['NUM', 'YY', 'MM', 'DD', 'HH24', 'weekday', 'week_name', 'STN',
                'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'elect']


    scaling_features = [feature for feature in selected_features if feature in S_columns]
    non_scaling_features = [feature for feature in selected_features if feature not in S_columns]

    #필요한 데이터만 추출
    scale_data = data[scaling_features].copy()
    non_scale_data = data[non_scaling_features].copy()

    #스케일러 적용
    scaled_R = scaler_R.fit_transform(scale_data)

    #결과를 다시 DataFrame으로 변환
    scaled_R = pd.DataFrame(scaled_R, columns=scale_data.columns)

    #데이터프레임 합치기
    final_data = pd.concat([scaled_R, non_scale_data.reset_index(drop=True)], axis=1)

    return final_data

In [ ]:
#sequence data
def create_sequence(data):
    seq_length=24
    num_sequences = len(data) - seq_length + 1

    #Numpy 배열을 미리 할당하여 메모리 사용을 최적화
    vs = np.zeros((num_sequences, seq_length, data.shape[1]))
    
    for i in tqdm(range(num_sequences), desc="Creating sequences"):
        vs[i] = data.iloc[i:i+seq_length].values

    return vs

In [ ]:
#tensor
def toTensor(vs):
    #입력값이 ndarray인지 확인
    if not isinstance(vs, np.ndarray):
        raise ValueError("Input data must be a numpy ndarray")
    
    # 데이터 복사를 피하고, float32로 변환
    tensor = torch.from_numpy(vs).float() 

    return tensor

## Preprocess

In [ ]:
#컬럼명 바꾸기
data = change_column_name(data)

In [ ]:
#시간 파생변수
time_data = preprocess_time(data)

In [ ]:
#3개월치 데이터 자르기
start_date = '2022-10-01 01:00:00'
end_date = '2023-01-01 00:00:00'
mask = (time_data['TM'] >= start_date) & (time_data['TM'] <= end_date)
filtered_data = time_data.loc[mask].copy()

In [ ]:
#결측치 처리
fillnan_data = fillnan(filtered_data)

In [ ]:
#계절 레이블 생성
seasonal_data = create_season_label(fillnan_data)

In [ ]:
#MAVG decomposition
mavg_data = moving_average_decomposition(seasonal_data)

In [ ]:
#차분값 파생변수
diffed_data = diff_data(mavg_data)

In [ ]:
#피처값 선택
non_summer_features = ['difference3_t', 'MM', 'nph_ta', 'DD', 'week_name', 'trend_s', 'difference2_e',
                       'difference3_h', 'HH24', 'difference2_h', 'seasonal_s', 'nph_ta_chi', 'residual_s', 'elect']

In [ ]:
#데이터 스케일링
scaled_data = scailing_data(diffed_data, non_summer_features)

In [ ]:
#sequence 생성
sequence_data = create_sequence(scaled_data)

In [ ]:
#tensor 변환
tensor_data = toTensor(sequence_data)
X_tensor = tensor_data

## y 데이터

In [ ]:
y = fillnan_data.loc[:, 'elect']

In [ ]:
y = pd.DataFrame(y)

In [ ]:
type(y)

In [ ]:
y_sequence = create_sequence(y)

In [ ]:
y_sequence_last = y_sequence[:, -1, :]  # 시퀀스의 마지막 값을 선택

In [ ]:
y_tensor = torch.tensor(y_sequence_last, dtype=torch.float32).unsqueeze(1).squeeze(1) # y_tensor의 크기를 (n, 1)로 변환

In [ ]:
y_tensor.size()

## TimeSeriesSplit을 이용한 교차검증

In [ ]:
#LSTM 모델 클래스 정의(드롭아웃 추가)
class LSTMmodel_non_summer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(LSTMmodel_non_summer, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

#하이퍼파라미터 설정
input_size = len(non_summer_features)
hidden_size = 100  # 은닉 상태 차원(100,150)
learning_rate = 0.0005 # 학습률(0.0005)
num_layers = 2  # LSTM 층의 수
output_size = 1  # 출력 차원
num_epochs = 100  # 학습 횟수
batch_size = 64  # 배치 크기
save_interval = 10  # 모델을 저장할 epoch 간격
dropout_prob = 0.3  # 드롭아웃 비율
save_dir = '../MODELS'  # 모델 저장 경로
early_stopping_patience = 10  # 조기 종료를 위한 인내 횟수

In [ ]:
#PyTorch 코드에서 GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# 교차 검증을 위한 TimeSeriesSplit 설정
tscv = TimeSeriesSplit(n_splits=5)

print(f'\nTraining and validating model for dataset with hidden size {hidden_size} and learning rate {learning_rate}')

# 교차 검증
for fold, (train_idx, val_idx) in enumerate(tscv.split(X_tensor)):
    print(f'Fold {fold+1}')
    X_train, X_val = X_tensor[train_idx], X_tensor[val_idx]
    y_train, y_val = y_tensor[train_idx], y_tensor[val_idx]

    # 데이터셋 생성
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    # 데이터 로더 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 모델 생성 및 디바이스 설정
    LSTMmodel = LSTMmodel_non_summer(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device) # GPU로 이동
    criterion = nn.MSELoss()  # 손실 함수: 평균 제곱 오차
    optimizer = optim.Adam(LSTMmodel.parameters(), lr=learning_rate)  # 옵티마이저: Adam

    # 상관계수를 저장할 리스트
    epoch_corrs = []
    best_val_loss = float('inf')
    early_stopping_counter = 0

    # 모델 훈련
    for epoch in tqdm(range(num_epochs), desc=f'Fold {fold+1} Epochs'):
        LSTMmodel.train()
        train_loss = 0
        for inputs, labels in tqdm(train_loader, desc='Training Batches', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            # 순전파 및 손실 계산
            outputs = LSTMmodel(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # 역전파 및 가중치 업데이트
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_loader)
        print(f'Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}')

        # 검증
        LSTMmodel.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in tqdm(val_loader, desc='Validation Batches', leave=False):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = LSTMmodel(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                all_preds.extend(outputs.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}')

        # 상관계수 계산
        all_preds = np.array(all_preds).flatten()
        all_targets = np.array(all_targets).flatten()
        corr, _ = pearsonr(all_preds, all_targets)
        epoch_corrs.append(corr)  # 상관계수를 리스트에 추가
        print(f'    Epoch {epoch+1}, Pearson Correlation: {corr:.4f}')

        # 조기 종료 체크
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
            # 모델 저장
            save_dir_epoch = os.path.join(save_dir, f'non_summer_hidden{hidden_size}_lr{learning_rate}')
            os.makedirs(save_dir_epoch, exist_ok=True)
            save_path = os.path.join(save_dir_epoch, f'fold_{fold+1}_epoch_{epoch+1}.pth')
            torch.save(LSTMmodel.state_dict(), save_path)
            print(f'Model saved to {save_path}')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= early_stopping_patience:
                print("Early stopping")
                break

    # 각 fold의 에포크 상관계수 평균 계산
    mean_corr = np.mean(epoch_corrs)
    print(f'    Fold {fold+1}, Mean Pearson Correlation: {mean_corr:.4f}')

print('All models trained and validated.')